In [ ]:
import os
import pandas as pd
import transformers as tr
from datasets import load_dataset


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Fill"

In [ ]:
!pip install --force-reinstall pydantic==1.10.6

In [ ]:
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [ ]:
import tempfile
tmpdir = tempfile.TemporaryDirectory()
local_training_root = tmpdir.name


In [ ]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9994"
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"


In [ ]:
imdb_ds = load_dataset("data.jsonl")

In [ ]:
model_name = "google/gemma-7b"

In [ ]:
tokenizer = tr.AutoTokenizer.from_pretrained(model_checkpoint, cache_dir=cache_dir)

In [ ]:
import torch

def to_tokens(tokenizer, label_map):
    def apply(x):
        target_labels = [label_map[y] for y in x["label"]]
        token_res = tokenizer(
            x["text"],
            text_target=target_labels,
            return_tensors="pt",
            truncation=True,
            padding=True,
        )
        # Convert tensors to lists or numpy arrays
        for key, value in token_res.items():
            if isinstance(value, torch.Tensor):
                token_res[key] = value.tolist()
        return token_res
    return apply

# Create function to convert IMDb dataset to tokens
imdb_to_tokens = to_tokens(tokenizer, imdb_label_lookup)

# Tokenize the IMDb dataset
tokenized_dataset = imdb_ds.map(
    imdb_to_tokens,
    batched=True,  # Expect the function to return a dictionary of types like (<class 'list'>, <class 'numpy.ndarray'>).
    remove_columns=["text", "label"]
)

def test_tokenized_dataset(tokenized_dataset, num_samples=1):
    # Print the first few samples from the tokenized dataset
    for i in range(num_samples):
        sample = tokenized_dataset[i]
        print(f"Sample {i + 1}:")
        print("Input IDs:", sample["input_ids"])
        print("Attention Mask:", sample["attention_mask"])
        print("Labels:", sample["labels"])
        print("=" * 50)

# Test the tokenized dataset
test_tokenized_dataset(tokenized_dataset['train'])

In [ ]:
# Define Zero configuration for optimization
zero_config = {
    "zero_optimization": {
        "stage": 2,
        "offload_optimizer": {"device": "cpu", "pin_memory": True},
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto",
            "torch_adam": True,
        },
    },
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto"
}

# Specify the model checkpoint to use
model_checkpoint = "base"

tokenizer = tr.AutoTokenizer.from_pretrained(
    model_checkpoint, cache_dir=DA.paths.datasets
)

# Convert IMDb dataset to tokens
imdb_to_tokens = to_tokens(tokenizer, imdb_label_lookup)
tokenized_dataset = imdb_ds.map(
    imdb_to_tokens, batched=True,
)

# Load the model for sequence-to-sequence learning
model = tr.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, cache_dir=DA.paths.datasets
)


In [ ]:
# Load the pre-trained model
model = tr.AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint,
    cache_dir=cache_dir
)

# Define checkpoint name and location
checkpoint_name = "test"
checkpoint_location = os.path.join(local_training_root, checkpoint_name)

# Define training arguments
training_args = tr.TrainingArguments(
    checkpoint_location,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    deepspeed=zero_config,
    report_to=["tensorboard"],
)


In [ ]:
# Define data collator
data_collator = tr.DataCollatorWithPadding(tokenizer=tokenizer)

# Create Trainer instance
trainer = tr.Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Define directory for TensorBoard display
tensorboard_display_dir = f"{checkpoint_location}/runs"


In [ ]:
# Load TensorBoard extension and display logs
%load_ext tensorboard
%tensorboard --logdir '{tensorboard_display_dir}'


In [ ]:
# Train the model
trainer.train()

# Save trained model to local checkpoint
trainer.save_model()
trainer.save_state()

# Persist the fine-tuned model to a directory
final_model_path = f"{cache_dir}/llm04_fine_tuning/{checkpoint_name}"
trainer.save_model(output_dir=final_model_path)


In [ ]:
# Load the fine-tuned model
fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)

# Generate predictions
inputs = tokenizer(
    reviews,
    return_tensors="pt",
    truncation=True,
    padding=True
)

pred = fine_tuned_model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"]
)

pred
